In [1]:
import MyNoteBook

In [2]:
import Valid_Datasets

In [3]:
instance = Valid_Datasets.get_df('00EABED2-271D-49D8-B599-1D4A09240601')

test_example = instance

cleaned_data = Valid_Datasets.non_watch_value_imputer(test_example)
cleaned_data = Valid_Datasets.KNN_for_watch_data(cleaned_data,10)
cleaned_data = Valid_Datasets.pca_to_data(cleaned_data,2)
cleaned_data


PCA explained variance ratio is 0.9390475853585886


,raw_acc:magnitude_stats:mean,raw_acc:3d:mean_x,raw_acc:3d:mean_y,raw_acc:3d:mean_z,proc_gyro:magnitude_stats:mean,proc_gyro:3d:mean_x,proc_gyro:3d:mean_y,proc_gyro:3d:mean_z,raw_magnet:magnitude_stats:mean,raw_magnet:3d:mean_x,raw_magnet:3d:mean_y,raw_magnet:3d:mean_z,watch_acceleration:magnitude_stats:mean,watch_acceleration:3d:mean_x,watch_acceleration:3d:mean_y,watch_acceleration:3d:mean_z,audio_naive:pc1,audio_naive:pc2
1444079161,0.996815,0.002331,0.004614,-0.996790,0.002558,0.000412,0.000448,-0.000541,618.751929,223.246192,97.503453,-568.776302,1032.508157,-0.592000,-55.824000,-1030.912000,-0.541354,-0.646983
1444079221,0.996864,0.003557,0.005495,-0.996832,0.002279,0.000091,-0.000082,0.000002,618.834334,223.362545,97.553554,-568.811684,1031.283026,0.000000,-56.096000,-1029.664000,2.820087,-1.764350
1444079281,0.996825,0.004180,0.005776,-0.996785,0.002579,0.000011,0.000179,0.000026,618.632716,223.252828,97.686277,-568.612604,1031.643125,2.912000,-57.696000,-1029.920000,3.283724,-1.735152
1444079341,0.996874,0.004179,0.005551,-0.996836,0.002497,-0.000006,-0.000020,0.000042,618.757055,223.281584,97.712646,-568.731966,1032.123385,2.640000,-56.400000,-1030.496000,0.640408,-0.925007
1444079431,0.997371,-0.008383,-0.000009,-0.994184,0.105566,-0.001074,-0.000934,0.006930,613.681143,86.943267,97.073641,-599.634130,1042.973796,66.458947,-101.187368,-1016.269474,8.601542,-0.563414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444750610,1.002052,0.004934,0.022278,-1.001781,0.002418,-0.000052,0.000125,0.000206,500.899097,76.519650,95.938905,-485.630765,1023.383704,3.996800,509.691200,-670.745600,2.528050,-1.362568
1444750670,1.002993,0.007210,-0.008207,-0.996744,0.126576,0.000681,-0.002937,-0.001118,506.277105,71.156095,117.209074,-486.893666,1022.034562,16.086400,526.772800,-687.646400,13.120239,0.417078
1444750731,1.002894,0.009963,-0.208783,-0.884378,0.351290,-0.015156,0.010175,0.020482,505.831997,73.140524,130.821746,-482.563144,1022.646160,0.955200,520.224000,-663.926400,8.691887,-0.640622
1444750768,1.002477,0.009740,-0.003631,-1.002408,0.003065,-0.000143,0.000361,-0.000267,518.997585,65.111670,145.811529,-493.818624,1020.716008,-9.876800,581.459200,-641.929600,7.597292,-1.084302


In [9]:
print(Valid_Datasets.get_cross_validation('test',0))

['0BFC35E2-4817-4865-BFA7-764742302A2D', '0E6184E1-90C0-48EE-B25A-F1ECB7B9714E', '1155FF54-63D3-4AB2-9863-8385D0BD0A13', '1DBB0F6F-1F81-4A50-9DF4-CD62ACFA4842', '27E04243-B138-4F40-A164-F40B60165CF3', '00EABED2-271D-49D8-B599-1D4A09240601', '098A72A5-E3E5-4F54-A152-BBDA0DF7B694', '0A986513-7828-4D53-AA1F-E02D6DF9561B', '11B5EC4D-4133-4289-B475-4E737182A406', '136562B6-95B2-483D-88DC-065F28409FD2', '1538C99F-BA1E-4EFB-A949-6C7C47701B20', '24E40C4C-A349-4F9F-93AB-01D00FB994AF']
